<a href="https://colab.research.google.com/github/mint26/IS5006_final/blob/main/IS5006_Group3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **IS5006 INTELLIGENT SYSTEMS DEPLOYMENT**

Final Group Project Title: Intelligent Multi Agent System with Case Based Reasoning<br>

Group 3<br>

Students Name: <br>
Ngan Zisheng Nigel<br>
Leong Zhong Wei Nicholas<br>
Jeremy Tan<br>
Tan Hui Min<br>
Ang Peng Chuan Alvin<br>
Tan Ming Yang Clarence<br>



# Install libraries

In [ ]:
!pip install PyDrive
!pip install 'gspread==3.7.0' 
!pip install 'gspread-dataframe==3.2.1' 
!pip install scikit_Fuzzy
!pip install ccxt
!pip install yfinance
!pip install schedule
!pip install panda

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1
  Found existing installation: gspread-dataframe 3.0.8
    Uninstalling gspread-dataframe-3.0.8:
      Successfully uninstalled gspread-dataframe-3.0.8
     |████████████████████████████████| 1.0MB 7.5MB/s 
  Created wheel for scikit-Fuzzy: filename=scikit_fuzzy-0.4.2-cp37-none-any.whl size=894069 sha256=65021d4eea00101d4c8224a94664298c0b1cfab78b84e90bbb0c897daf9bd2cc
  Stored in directory: /root/.cache/pip/wheels/b9/4e/77/da79b16f64ef1738d95486e2731eea09d73e90a72465096600
Successfully built scikit-Fuzzy
     |████████████████████████████████| 2.1MB 8.8MB/s 
     |████████████████████████████████| 1.3MB 44.0MB/s 
     |████████████████████████████████| 163kB 47.5MB/s 
     |████████████████████████████████| 3.2MB 45.3MB/s 
     |████████████████████████████████| 143kB 48.6MB/s 
     |████████████████████████████████| 235kB 46.5MB/s 
  Created wheel for yarl: filenam

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import json
import time
import datetime
from datetime import datetime, timedelta, date
from functools import reduce

import ccxt
import random
import requests
import math
import os.path
import re
import pickle
from googleapiclient.discovery import build

import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from oauth2client.client import GoogleCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

from oauth2client.client import GoogleCredentials
from google.colab import auth
from google.colab import drive
from google.cloud import language_v1
from google.cloud.language_v1 import enums

import pandas_datareader.data as web
from threading import Lock, Thread
from multiprocessing import Process, Queue, Manager
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from collections import defaultdict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from scipy.spatial.distance import cdist
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, classification_report, roc_curve, auc

import tweepy
import re
from textblob import TextBlob

In [ ]:
# These are constants needed for the functions to run
Project_Start_Date = datetime.now().strftime("%Y/%m/%d")
Recommendation_ID = 0
Short_Period = 20

# Training

# Actual Application

## Constants
The Constants object stores the common static variables used by the different agents. 




In [ ]:
class Constants:
  DATASET01 = 'dataset01' 
  DATASET02 = 'dataset02'
  DATABASE = 'IS5006_DB'
  EOS = 'EOS' 
  ETH = 'ETH'
  EOSMA = 'EOSMA'
  ETHMA = 'ETHMA'
  STOPORDER = 'StopOrders'
  EOSDatabase = 'EOSDatabase'
  ETHDatabase = 'ETHDatabase'
  PNL = 'PnL'
  ORDERS = 'Orders'
  EOSMODEL = 'EOSModel' 
  ETHMODEL = 'ETHModel'


## Data Agent
The data agent is responsible for getting the latest data from Bitfinex and store it to google spreadsheet for other agents to do analysis. On top of that, the data agent provides read and write functions from the shared google spreadsheet so that it could be utilized by other agents. 

* `getCandleFromBitfinex` contains the implementation to pull the data from the
Bitfinex and it supports different intervals as well different coins. A simple input validation is in place to ensure the correct parameters are being passed by the caller function. In each api call, maximum of 10000 record can be retrieved and its sorted by date in ascending order. The output of this function is the last record date so that data can be pulled continuously till today date. 
* `pull_coin` - Get the latest data from bitfinex for different symbol by calling `getCandleFromBitfinex`. In our current setup, the two cryptocurrencies used will be EOS and ETH. 
*   `read_worksheet_to_df` - Get the data from specific worksheet stores in `IS5006_DB`, such that dataframe can be returned to the calling agent. 
*   `write_to_ma_worksheet` - A helper write function that is used by Crossover agent to write the moving average result into the `EOSMA`/`ETHMA` worksheets in `IS5006_DB` file. 
*   `write_to_specific_worksheet` - Another helper write function for other agent to write their result into the specific worksheet in `IS5006_DB`
* `getInstance` - The data agent is a singleton object to ensure consistent data when reading by different agents in multithread environment. 
* `cleanup` - This method closes the client connection and recreate a new data agent object such that new connection could be reconnected due to timeout issue from google spreadsheet api.





In [ ]:
class DataAgent:
  __instance = None
  eos_worksheet = None
  eth_worksheet = None
  btc_worksheet = None
  eos_ma_worksheet = None
  eth_ma_worksheet = None
  stop_order_worksheet = None
  eos_db_worksheet = None 
  eth_db_worksheet = None
  eos_model_worksheet = None 
  eth_model_worksheet = None
  pnl_worksheet = None
  orders_worksheet = None
  scope = "" 
  sheet = None
  start_date = datetime(year=2021, month=1, day=1, hour=0, second=0)

  def init_oldest_date(self, date): 
    self.start_date = date

  def getCandleFromBitfinex(self, worksheet, duration, coinSymbol, latest, start, end): 
    supported_intervals = {'1m', '5m', '15m', '30m', '1h', '3h', '6h', '12h', '1D', '7D', '14D', '1M'}
    assert duration in supported_intervals,\
      'duration should be one of %s' % supported_intervals

    supported_coins = {'tETHUSD','tEOSUSD','tBTCUSD'}
    assert coinSymbol in supported_coins,\
      'coin symbol should be one of %s' % supported_coins

    section = 'hist'
    if latest: 
      section = 'last'

    print(start)
    print(end)
    startStr = str(math.trunc(start.timestamp() * 1000))
    endStr = str(math.trunc(end.timestamp() * 1000))
    if startStr == endStr:
      url =  '%s%s%s%s' % ('https://api-pub.bitfinex.com/v2/candles/',
                                'trade:' + duration + ':' + coinSymbol,
                                '/' + section + '?start=' + startStr, 
                                '&limit=10000&sort=1')
    else:
      url =  '%s%s%s%s%s' % ('https://api-pub.bitfinex.com/v2/candles/',
                                      'trade:' + duration + ':' + coinSymbol,
                                      '/' + section + '?start=' + startStr, 
                                      '&end=' + endStr, 
                                      '&limit=10000&sort=1')
    response = requests.get(url)
    data = response.json()
    # If data is not available, we skip the proccesing
    if len(data) <= 0:
      return start
    df = pd.DataFrame.from_records(data)


    df[0] = pd.to_datetime(df[0], unit='ms')
    df[0] = df[0].dt.strftime('%Y-%m-%d %H:%M:%S')
    df = df.rename(columns={0: "Date",1:"Open",2:"Close", 3:"High", 4:"Low", 5:"Volume"})
    last_row = df.tail(1)
    end_date = last_row['Date'].values[0]
    idx = pd.date_range(start, end_date, freq="5min").strftime('%Y-%m-%d %H:%M:%S')
    df.set_index('Date').reindex(idx).reset_index()
    worksheet.append_rows(df.values.tolist())
    return end_date

  def pull_coin(self, worksheet, coin): 
    start = self.start_date
    days = timedelta(31)
    end = start + days
    today = datetime.now()
    if end > today: 
        end = today

    row_count = worksheet.row_count
    values_list = worksheet.row_values(row_count)
    if row_count > 1:
      np_arr = np.array(values_list)
      start = datetime.strptime(np_arr[0], '%Y-%m-%d %H:%M:%S')
    if start + timedelta(seconds=300) > today: 
      start = today - timedelta(seconds=300)
    isTerminate = True
    while isTerminate:
      if isinstance(start, str):
        start = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
      if row_count > 1: 
        start = start + timedelta(seconds=300)
      days = timedelta(31)
      end = start + days
      if end > today: 
        isTerminate = False
        end = today
      print("extracting data from " + start.strftime("%m/%d/%Y, %H:%M:%S") + " to " + end.strftime("%m/%d/%Y, %H:%M:%S"))
      start = self.getCandleFromBitfinex(worksheet,'5m', coin, False,start, end)

  def execute(self): 
    print("=========Pulling EOS data========")
    self.pull_coin(self.eos_worksheet,'tEOSUSD')
    print("=========Pulling ETH data========")
    self.pull_coin(self.eth_worksheet,'tETHUSD')
    # print("=========Pulling BTC data========")
    # self.pull_coin(self.btc_worksheet,'tBTCUSD')

  def read_worksheet_to_df(self, worksheet):
    df = get_as_dataframe(worksheet)
    return df

  def write_to_ma_worksheet(self, df,worksheet):
    df['Date'] = pd.to_datetime(df['Date'], unit='ms',errors='coerce')
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df = df.replace(np.nan, 'NAN', regex=True)
    data = df.values.tolist()
    worksheet.append_rows(data)

  def write_to_specific_worksheet(self, worksheet, data): 
    worksheet.append_rows(data)

  def connect(self, key):
    scope = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(key, scope)
    client = gspread.authorize(credentials)
    self.sheet = client.open(Constants.DATABASE)
    worksheet_list = self.sheet.worksheets()
    self.eos_worksheet = self.sheet.worksheet(Constants.EOS)
    self.eth_worksheet = self.sheet.worksheet(Constants.ETH)
    self.eos_ma_worksheet = self.sheet.worksheet(Constants.EOSMA)
    self.eth_ma_worksheet = self.sheet.worksheet(Constants.ETHMA)
    self.stop_order_worksheet = self.sheet.worksheet(Constants.STOPORDER)
    self.eos_db_worksheet = self.sheet.worksheet(Constants.EOSDatabase)
    self.eth_db_worksheet = self.sheet.worksheet(Constants.ETHDatabase)
    self.pnl_worksheet = self.sheet.worksheet(Constants.PNL)
    self.orders_worksheet = self.sheet.worksheet(Constants.ORDERS)

  def __init__(self, key):
    self.connect(key)
    # self.btc_worksheet = sheet.worksheet(Constants.BTC)
    if DataAgent.__instance != None:
        raise Exception("Only single data agent can be created")
    else:
        DataAgent.__instance = self 
  @staticmethod 
  def getInstance(key = '/content/client_secret.json'):
    if not DataAgent.__instance:
      DataAgent(key)
    return DataAgent.__instance

  @staticmethod 
  def cleanup():
    DataAgent.__instance = None  
  

## Crossover Agent

Crossover agent is one of the N agents that does its own analysis before passing the output to the Decider agent for further analysis. In this agent, it gets the candlestick data in dataframe from Data Agent via `read_worksheet_to_df` and performs weighted moving average calculation on the provided data. After getting the figures, a fuzzy logic is implemented to determine the signal for the Decider Agent. 


*   `get_historical_ma` - used to get the historical moving average for the historical data required to train the KNN models. 
*   `plot_crossover_graph` - a visualization helper function to illustrate how the signal would look like in 5 minute interval. 
* `set_optimal_period` - used to evaluate the optimal short period by checking on the p value to see if the dataset is overfitting. 
* `run` - is the function execute when doing live trading which it will constantly compute the moving average at every interval. 



In [ ]:
class CrossoverAgent:
  selected_coin = None
  short_period = Short_Period 
  long_period = short_period * 2
  eos_ma_df = pd.DataFrame()
  eth_ma_df = pd.DataFrame()
  data_agent = None

  def run(self, eos_result, eth_result):
    data_agent = DataAgent.getInstance()
    # data_agent.execute()
    print ("Doing crossover analysis")
    eos_df = data_agent.read_worksheet_to_df(data_agent.eos_worksheet)
    self.eos_ma_df = pd.DataFrame(eos_df[['Date','Close']])
    self.eos_ma_df['Symbol'] = Constants.EOS
    eth_df = data_agent.read_worksheet_to_df(data_agent.eth_worksheet)
    self.eth_ma_df = pd.DataFrame(eth_df[['Date','Close']])
    self.eth_ma_df['Symbol'] = Constants.ETH
    self.update_crossover(self.eos_ma_df)
    self.update_decision(eos_result, self.eos_ma_df)
    self.update_crossover(self.eth_ma_df)
    self.update_decision(eth_result, self.eth_ma_df)

  def train(self): 
    data_agent = DataAgent.getInstance()
    # data_agent.execute()
    print ("Doing crossover analysis")
    eos_df = data_agent.read_worksheet_to_df(data_agent.eos_worksheet)
    self.eos_ma_df = pd.DataFrame(eos_df[['Date','Close']])
    self.eos_ma_df['Symbol'] = Constants.EOS
    eth_df = data_agent.read_worksheet_to_df(data_agent.eth_worksheet)
    self.eth_ma_df = pd.DataFrame(eth_df[['Date','Close']])
    self.eth_ma_df['Symbol'] = Constants.ETH
    self.update_crossover(self.eos_ma_df)
    self.update_crossover(self.eth_ma_df)
    self.selected_coin = Constants.EOS
    self.plot_crossover_graph(self.eos_ma_df)
    self.selected_coin = Constants.ETH
    self.plot_crossover_graph(self.eth_ma_df)
    self.set_optimal_period(self.eos_ma_df)
    self.set_optimal_period(self.eth_ma_df)
    
  def get_historical_ma(self, symbol): 
    data_agent = DataAgent.getInstance()
    ma_df = self.eth_ma_df
    worksheet = data_agent.eth_ma_worksheet
    if symbol == Constants.EOS:
      ma_df = self.eos_ma_df 
      worksheet = data_agent.eos_ma_worksheet
    start = datetime.datetime(year=2021,month=3,day=1,hour=0, second=0)
    delta = datetime.timedelta(seconds=20000)
    start = start - delta
    ma_df['Date'] = pd.to_datetime(ma_df['Date'])
    end = datetime.datetime(year=2021, month=4, day=1, hour=0, second=0)
    mask = (ma_df['Date'] >= start) & (ma_df['Date'] <= end)
    chunk_eth_df = ma_df.loc[mask]
    output_df = self.update_crossover(chunk_eth_df)
    data_agent.write_to_ma_worksheet(output_df,worksheet)

  def update_crossover(self, ma_df):
    short_wts = np.arange(1,self.short_period + 1) 
    long_wts = np.arange(1, self.long_period + 1)
    ma_df['SMA'] = ma_df['Close'].rolling(self.short_period).apply(lambda prices: np.dot(prices, short_wts)/short_wts.sum(), raw=True)
    ma_df['LMA'] = ma_df['Close'].rolling(self.long_period).apply(lambda prices: np.dot(prices, long_wts)/long_wts.sum(), raw=True)
    ma_df['Ratio'] = ma_df['SMA'] / ma_df['LMA']
    ma_df['Signal'] = np.where(ma_df['SMA'] > ma_df['LMA'], 1.0, 0.0)
    ma_df['Position'] = ma_df['Signal'].diff()
    return ma_df

  def plot_crossover_graph(self, ma_df): 
    title = 'EOS Strategy'
    if self.selected_coin == Constants.ETH:
      title = 'ETH Strategy'
    #Signals graph
    plt.figure(figsize = (20,10))
    df = ma_df.iloc[-100:].copy()
    # plot close price, short-term and long-term moving averages 
    df['Close'].plot(color = 'k', label= 'Close Price') 
    df['SMA'].plot(color = 'r',label = 'SMS') 
    df['LMA'].plot(color = 'g', label = 'LMA')
    # plot ‘buy’ signals
    plt.plot(df[ma_df['Position'] == 1].index, 
            df['SMA'][df['Position'] == 1], 
            '^', markersize = 15, color = 'g', label = 'buy')
    # plot ‘sell’ signals
    plt.plot(df[ma_df['Position'] == -1].index, 
            df['SMA'][df['Position'] == -1], 
            'v', markersize = 15, color = 'r', label = 'sell')
    plt.ylabel('USD', fontsize = 15 )
    plt.xlabel('Date', fontsize = 15 )
    plt.title(title, fontsize = 20)
    plt.legend()
    plt.grid()
    plt.show()

  def set_optimal_period(self, ma_df): 
    result = []
    train_size = 0.6
    n_forward = 10
    ma_df['Forward Close'] = ma_df['Close'].shift(-n_forward)
    ma_df['Forward Return'] = (ma_df['Forward Close'] - ma_df['Close'])/ma_df['Close']
    ma_df['current_input'] = [int(x) for x in ma_df['Close'] > ma_df['SMA']]
    cur_df = ma_df.dropna()
    cur_training = cur_df.head(int(train_size * cur_df.shape[0]))
    cur_test = cur_df.tail(int((1 - train_size) * cur_df.shape[0]))
    tr_returns = cur_training[cur_training['current_input'] == 1]['Forward Return']
    test_returns = cur_test[cur_test['current_input'] == 1]['Forward Return']
    cur_mean_forward_return_training = tr_returns.mean()
    cur_mean_forward_return_test = test_returns.mean()
    cur_pvalue = ttest_ind(tr_returns,test_returns,equal_var=False)[1]
    current_result = {
        'sma_length': self.short_period,
        'training_forward_return': cur_mean_forward_return_training,
        'test_forward_return': cur_mean_forward_return_test,
        'p-value': cur_pvalue
    }
    for sma_length in range(5,200):
      ma_df['input'] = [int(x) for x in ma_df['Close'] > ma_df['SMA']]
      
      df = ma_df.dropna()
      training = df.head(int(train_size * df.shape[0]))
      test = df.tail(int((1 - train_size) * df.shape[0]))
      
      tr_returns = training[training['input'] == 1]['Forward Return']
      test_returns = test[test['input'] == 1]['Forward Return']
      mean_forward_return_training = tr_returns.mean()
      mean_forward_return_test = test_returns.mean()
      pvalue = ttest_ind(tr_returns,test_returns,equal_var=False)[1]
      result.append({
          'sma_length':sma_length,
          'training_forward_return': mean_forward_return_training,
          'test_forward_return': mean_forward_return_test,
          'p-value':pvalue
      })
      result.sort(key = lambda x : -x['training_forward_return'])
    if result[0]['p-value'] < current_result['p-value']: 
      print(result[0])
      print('new optimal short period for '+ self.selected_coin + ' is ' + str(result[0]['sma_length']))
      self.set_short_period(result[0]['sma_length'])
    else: 
      print(current_result)
      print('new optimal short period for '+ self.selected_coin + ' is ' + str(current_result['sma_length']))

  def update_decision(self,result,ma_df): 
    decision = 'hold'
    last_row = ma_df.tail(1)
    close = last_row['Close'].values[0]
    position = last_row['Position'].values[0]
    symbol = last_row['Symbol'].values[0]
    result['MAAgent'] = position

    result['Current_Price'] = close
    print("MA Agent's Output To Google Sheets")
    print("Close Price", close, "MA Agent", position)

    if position == -1.0:
      decision = 'sell'
    elif position == 1.0:
      decision = 'buy'

  def set_short_period(self, suggested_period): 
    global Short_Period
    Short_Period = suggested_period
    self.long_period = Short_Period * 2
    


Before live trading begins, the `train` function is invoked to find the optimal n period by executing the following steps in `set_optimal_period` function.
1. Split this dataset into training and test sets
2. Apply different moving averages on the training set and, for each one, calculate the average return value after N days when the close price is over the moving average 
3. Choose the moving average length that maximizes such average return
Use this moving average to calculate the average return on the test set
4. Verify that the average return on the test set to perform cross validation, ensuring there isnt an overfitting situation. 

By default, the short period will be set to 20. If there is a better SMA period lower than the p-index for 20-period, we will then set the optimal period for the crossover agent to that value. 



## Commodities Agent

We research on news and online articles on suggestions which commodities and stocks have good correlations with Cryptocurrencies and found out that the following is good:
Soybean Futures, Sugar, S&P500, QQQ ETF, Vaneck Semiconductor ETF, Dow Jones Industrial, MicroStrategy Incorporated (bitcoin orientated company), Riot Blockchain Inc, Paypal, Master, TSLA, Chicago Mercantile Exchange, Square, IBm and Amazon.
Based on their direction of percentage price change in the past 5mins closed, we predict what is the direction of percentage price change of ETH and EOS in the next 5mins. We use Machine Learning Decision Tree Classifier to get an output of a BUY/SELL signal for ETH/EOS. The output is a one-hot state of 1 mapping to a positive direction and -1 mapping to a negative direction.

To read more about correlation between cryptocurrencies and stocks refer to these online resources
* https://www.forbes.com/sites/investor/2020/05/13/bitcoin-and-stocks-correlation-reveal-a-secret/?sh=25386a9b12c2
* https://www.fool.com/investing/stock-market/market-sectors/financials/cryptocurrency-stocks/
* https://www.nasdaq.com/articles/top-blockchain-stocks-to-buy-right-now-4-names-for-your-list-2021-03-30
* https://www.nasdaq.com/articles/making-a-list-of-the-top-cryptocurrency-stocks-to-watch-4-names-to-know-2021-03-02
* https://www.forbes.com/sites/greatspeculations/2021/02/19/with-bitcoin-at-50k-which-crypto-stocks-should-you-pick/?sh=57498adb7203 <br><br>



> Bonus : We use sklearn Decision Tree Machine Learning to predict the signal direction of the cryptocurrency <br>
https://blog.quantinsti.com/decision-tree/ <br>





In [ ]:
class CommoditiesAgent: 
  def __init__(self, eos_result, eth_result):
    symbol_list = ['ETH-USD','EOS-USD','ZS=F', 'SB=F',  'SPY', 'QQQ', 'SMH', '^DJI', 'MSTR', 'RIOT',  'PYPL', 'MA', 'TSLA', 'CME', 'SQ', 'IBM', 'AMZN']
    #Import the decision tree model for ETH
    decision_tree_model_pkl_ETH = open('/content/decision_tree_regressor_ETH.pkl', 'rb')
    decision_tree_model_ETH = pickle.load(decision_tree_model_pkl_ETH)
    #print(decision_tree_model_ETH)
    #Import the decision tree model for EOS
    decision_tree_model_pkl_EOS = open('/content/decision_tree_regressor_EOS.pkl', 'rb')
    decision_tree_model_EOS = pickle.load(decision_tree_model_pkl_EOS)
    #print(decision_tree_model_EOS)
    # TODO: result for decider
    results=self.fetch_commodities_live(symbol_list,decision_tree_model_ETH,decision_tree_model_EOS, eos_result, eth_result)
    ETH_predicted_direction=results[0]
    EOS_predicted_direction=results[1]
    #print(ETH_predicted_direction)
    #print(EOS_predicted_direction)

  def fetch_commodities(self,symbol,today):
    # Stock market details for the symbol we wish to analyze
    stock = yf.Ticker(symbol)
    # Check the stock info. If there is no stock information available, there will be a warning generated.
    #print(stock.info)
    time.sleep(1)
    #dummy = stock.info
    # Fetch the 5min interval 
    stock_5min = stock.history(period="5d",interval='5m',start=today)
    # Leave the datetime in UTC format for consistency across all agents
    commodity = stock_5min.tz_convert(None)
    return commodity

  def sign(self,number) :
    if number >= 0 :
      return 1
    else :
      return -1
  def fetch_commodities_live(self,symbol_list,decision_tree_model_ETH,decision_tree_model_EOS, eos_result, eth_result):
    df = pd.DataFrame()
    #Current UTC time - 5days ago
    dt=datetime.utcnow() - timedelta(days=5)
    today=dt.strftime("%Y-%m-%d")
    for sym in symbol_list :
      #Get 5 days worth of datasets
      tmp=self.fetch_commodities(sym,today)
      df[sym] = tmp['Close']
    # Fill up NA datasets with previous data. 
    df=df.fillna(method='ffill')
    # Drop the NA rows which are redundant.
    df=df.dropna()
    # Obtain latest 2 datasets which at 5minutes apart
    df=df.tail(2)
    # Obtain the percentage price change
    df=df.pct_change()
    # change the percentage price change to onehot. 1 for positive direction signal and -1 for negative direction signal
    df=df.applymap(lambda x: self.sign(x))
    df=df.tail(1)
    df=df.iloc[:,2:17]
    # Obtain the input of the dataset for the deicision tree model
    X=df
    # Predict the direction of the signal using decision tree regression model 
    y_pred_EOS = decision_tree_model_EOS.predict(X)
    y_pred_ETH = decision_tree_model_ETH.predict(X)
    eos_result['CommoditiesAgent'] = y_pred_EOS
    eth_result['CommoditiesAgent'] = y_pred_ETH
    if isinstance(y_pred_EOS,(np.ndarray)):
      y_pred_EOS=y_pred_EOS[0]
      y_pred_ETH=y_pred_ETH[0]
    else:
      pass
    print("Commodities Agent's Output To Google Sheets")
    print("ETH", float(y_pred_ETH), "EOS", float(y_pred_EOS))
    return float(y_pred_ETH),float(y_pred_EOS)


## Twitter Agent

For the twitter agent, it will search on twitter using tweepy API for specific words and words with hashtags relating to the cryptocurrency of interest. Once it find the recent 100 tweets, it will take the tweets within the last 30 seconds based on their timestamps and preprocess the tweets to clean unnecessary data. Then sentiment analysis is performed on the cleaned tweet texts to obtain the average polarity score and average subjectivity score using the `fetch_tweets` functions.

> Bonus : We use polarity score and subjective score for twitter sentiment analysis instead of the usual NLP crisp values. <br>
https://www.analyticsvidhya.com/blog/2018/02/natural-language-processing-for-beginners-using-textblob/

In [ ]:
class TwitterAgent():
  def clean_tweet_text(self,tweetText):
    # Change all characters to lower case
    tweetText = tweetText.lower() 
    # Remove any strings starting with a "@"
    tweetText = re.sub("@[A-Za-z0-9]+", "", tweetText) 
    # Remove any strings starting with a "#"
    tweetText = re.sub("#[A-Za-z0-9]+", "", tweetText) 
    # Remove any strings starting with a "$"
    tweetText = re.sub("\$[A-Za-z]+", "", tweetText) 
    # Remove the "\n" string
    tweetText = re.sub("\\n", " ", tweetText) 
    # Remove any hyperlinks
    tweetText = re.sub("https?:\/\/\S+", "", tweetText) 
    # Remove url/website that didn't use http, is only checking for .com websites, so words that are seperated by a . are not removed
    tweetText = re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', tweetText)
    # Remove {link}
    tweetText = re.sub(r'{link}', '', tweetText) 
    # Remove &text; html chars
    tweetText = re.sub(r'&[a-z]+;', '', tweetText) 
    # Remove [video]
    tweetText = re.sub(r"\[video\]", '', tweetText) 
    # Remove all remaining characters that aren't letters, white space, or  the following #:)(/\='] that are used in emojis or hashtags
    tweetText = re.sub(r"[^a-z0-9\s\(\-:\)\\\/\];='#$]", '', tweetText) 
    # Return the cleaned tweet text to the caller
    return tweetText
    
  def fetch_tweets(self, timeNow, cryptoName, result):
    # Set Twitter API credentials
    consumerKey = '9jOeKyJbRcmdyiytKCKgb2xhB'
    consumerSecret = 'AqVpKYq0s7rsUs2NBOf9n9YXix33HFzV1I91MhfjiuhgxQ4upM'
    accessToken = '1370716861860433921-vCIVAOvYRjQhEuD6vY6qsoJvGMeS8X'
    accessTokenSecret = 'UpxWn8yIW3OGUxt93SerTAxE0aTvk5dy8Fgp9pqRy9FDE'

    # Create the authentication object
    authentication = tweepy.OAuthHandler(consumerKey, consumerSecret)
    # Set the access token and the access token secret
    authentication.set_access_token(accessToken, accessTokenSecret)
    # Create the API object
    api = tweepy.API(authentication, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

    # Declare secondsBefore as variable to store 30 as integer 
    secondsBefore = 30
    # Declare timeBefore as datetime variable to store the time 30 seconds before in UTC
    timeBefore = timeNow - timedelta(0, secondsBefore)

    # Declare searchEos as string variable to store string for searching in twitter if targeted cryptocurrency is EOS/USDT (refer to above for more information)
    searchEos = "(#eosusdt OR #eos) (profit OR recommendation OR lose OR gain OR lost OR short OR long OR hold OR hodl OR trade OR buy OR sell OR loss OR price OR climb OR drop)"
    # Declare searchEth as string variable to store string for searching in twitter if targeted cryptocurrency is ETH/USDT (refer to above for more information)
    searchEth = "(#ethusdt OR #eth OR #ethereum) (profit OR recommendation OR lose OR gain OR lost OR short OR long OR hold OR hodl OR trade OR buy OR sell OR loss OR price OR climb OR drop)"

    if cryptoName == "ETH":
        # Declare tweets as a cursor object to gather 100 tweets in english language based on the search string for ETH/USDT
        tweets = tweepy.Cursor(api.search, q = searchEth, lang="en", tweet_mode = "extended").items(100)
    elif cryptoName == "EOS":
        # Declare tweets as a cursor object to gather 100 tweets in english language based on the search string for EOS/USDT 
        tweets = tweepy.Cursor(api.search, q = searchEos, lang="en", tweet_mode = "extended").items(100)

    # Declare tweetTexts as list variable to store recent 100 tweet texts
    tweetTexts = []
    # Declare timeStamps as list variable to store recent 100 timestamps
    timeStamps = []
    # Iterate through the list of tweets
    for tweet in tweets:
        # Add tweet text to tweetTexts 
        tweetTexts.append(tweet.full_text)
        # Add timestamp to timeStamps 
        timeStamps.append(tweet.created_at)

    # Declare selectedTweetTexts as list variable to store tweet texts that was created in the last 30 seconds
    selectedTweetTexts = []
    # Iterate through the list of tweets
    for index in range(len(timeStamps)):
        # Declare timeStamp as variable to store the timestamp of the tweet
        timeStamp = timeStamps[index]
        # Declare tweetText as variable to store the text of the tweet
        tweetText = tweetTexts[index]
        # If the timestamp for the tweet text is within 30 seconds this function is called, add the tweet text to selectedTweetTexts
        if timeStamp >= timeBefore and timeStamp <= timeNow:
            selectedTweetTexts.append(tweetText)

    # Declare polarityScores as list variable to store polarity score of each tweet text that was created in the last 30 seconds
    polarityScores = []
    # Declare subjectivityScores as list variable to store subjectivity score of each tweet text that was created in the last 30 seconds
    subjectivityScores = []
    # if there are tweets created in the last 30 seconds based on the string search in twitter,  
    if len(selectedTweetTexts) > 0:
        for index in range(len(selectedTweetTexts)):
            # Declare selectedTweetText as variable to store the raw tweet text
            selectedTweetText = selectedTweetTexts[index]
            # Declare selectedCleanTweetText as variable to store the cleaned tweet text
            selectedCleanTweetText = self.clean_tweet_text(selectedTweetText)
            # Declare selectedCleanTweetText as variable to store the polarity score for the cleaned tweet text
            polarityScores.append(TextBlob(selectedCleanTweetText).sentiment.polarity)
            # Declare selectedCleanTweetText as variable to store the subjectivity score for the cleaned tweet text
            subjectivityScores.append(TextBlob(selectedCleanTweetText).sentiment.subjectivity)
        # Declare numberTweets as variable to store the total number of tweets in the last 30 seconds
        numberTweets = len(polarityScores)
        # Declare averagePolarityScore as variable to store the average polarity score in the last 30 seconds
        averagePolarityScore = sum(polarityScores)/numberTweets
        # Declare averageSubjectivityScore as variable to store the average subjectivity score in the last 30 seconds
        averageSubjectivityScore = sum(subjectivityScores)/numberTweets
    else:
        # Set the number of tweets, average polarity score and average subjectivity score to 0 if there are no tweets in the last 30 seconds
        numberTweets = 0
        averagePolarityScore = 0
        averageSubjectivityScore = 0
    print("Twitter Agent's Output To Google Sheets")
    print("averagePolarityScore",averagePolarityScore, "averageSubjectivityScore", averageSubjectivityScore)
    result['AveragePolarity'] = averagePolarityScore
    result['AverageSubjectivity'] = averageSubjectivityScore
    # Return the number of tweets, average polarity score and average subjectivity score to the caller
    return numberTweets, averagePolarityScore, averageSubjectivityScore


## Decider Agent

The Decider Agent takes in inputs from the N Agents and outputs the buy/sell signal and Kelly Criterion of the current model by invoking the `run_latest_model` function. The model is trained over all the previously recorded cases.

> Bonus : We use Kelly Criterion for bet sizing. This is a mathematical formulae used frequently by investors and gamblers. It calculates and decides how much money they should allocate to each investment or bet through a predetermined fraction of assets. <br>
https://corporatefinanceinstitute.com/resources/knowledge/trading-investing/kelly-criterion/#:~:text=Summary-,Kelly%20criterion%20is%20a%20mathematical%20formula%20for%20bet%20sizing%2C%20which,a%20predetermined%20fraction%20of%20assets. <br>

> Bonus : We also use KNN as our CBR instead of conventional CBR models since KNN is both a Lazy Algorithm and also a Case-Based Learning Algorithm. KNN is also a versatile supervised machine learning algorithm. We can use KNN for both regression and classification problems. <br>
https://medium.com/@springboard_ind/knn-machine-learning-algorithm-explained-596d60336076 <br>


In [ ]:
class DeciderAgent():
  # Class Variables
  features = ['MAAgent', 'AverageSubjectivity','AveragePolarity','CommoditiesAgent']
  label = ['Label']
  closeprice = ['ClosePrice']
  training_range = range(1,10)

  # function to pull cases out from google sheets
  def get_database(symbol):
    data = wb.worksheet(symbol+'Database')
    data = data.get_all_values() 
    database = pd.DataFrame(data)
    new_header = database.iloc[0]
    database = database[1:]
    database.columns = new_header
    
    # data_agent = DataAgent.getInstance()
    # if symbol == Constants.ETH:
    #  database = data_agent.read_worksheet_to_df(data_agent.eth_db_worksheet)
    # else: 
    #  database = data_agent.read_worksheet_to_df(data_agent.eos_db_worksheet)

    input = database[DeciderAgent.features]
    output = database[DeciderAgent.label]
    closeprice = database[DeciderAgent.closeprice]

    return input, output, closeprice

  # function that gives the Kelly Criterion and Buy/Sell signal based on N agents input
  def run_latest_model(symbol,data_input,price,decider_result):

    print('######### Decider Agent Start #########')

    # Starttime of function  
    start_time = datetime.now().strftime("%B %d, %Y, %H:%M:%S")
    
    # Training the KNN model based on historical cases
    # Checklist: Syncronize the agent response so that all agents provide data one on one tick for review
    input = DeciderAgent.get_database(symbol)[0]
    input = input.astype(float)
    output = DeciderAgent.get_database(symbol)[1]
    output = output.astype(int)
    
    # Checklist: Remember and rank previous decisions
    # Checklist: Give weightage to the decision based on the P&L report
    preprocess = make_column_transformer(
          (make_pipeline(
          StandardScaler()), DeciderAgent.features),
      )
    pipeline = make_pipeline(preprocess,KNeighborsClassifier())
    params = {'kneighborsclassifier__n_neighbors': DeciderAgent.training_range}
    X_train, X_test, y_train, y_test = train_test_split(input, output, test_size=0.3, random_state=642)
    model = GridSearchCV(pipeline, params, cv=5, scoring='precision')
    model.fit(X_train, y_train.values.ravel())
    k = model.best_params_['kneighborsclassifier__n_neighbors']
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(str(symbol) + ' model Accruacy score is: ' + str(accuracy))
    precision = precision_score(y_test, y_pred)
    print(str(symbol) + ' model Precision score is: ' + str(precision))
    predictions = model.predict(input)

    # Checklist: Case Based Reasoning: Suggests quantity
    # Updating the model's win percentage and loss percentage over historical cases to calculate Kelly's Criterion
    closeprice = DeciderAgent.get_database(symbol)[2]
    closeprice = closeprice.to_numpy()
    closeprice = closeprice.astype(float)
    output = output.to_numpy()

    winpercentage=0
    wincount=0
    losepercentage=0
    losecount=0

    for i in range(len(predictions)):
      if i == len(predictions)-1:
        pass
      else:  
        if output[i][0] == predictions[i]:
          winpercentage += abs(closeprice[i+1][0]-closeprice[i][0])/closeprice[i][0]
          wincount += 1
        else:
          losepercentage += abs(closeprice[i+1][0]-closeprice[i][0])/closeprice[i][0]
          losecount += 1

    winpercentage = winpercentage/wincount
    losepercentage = losepercentage/losecount
    kelly = accuracy - ((1-accuracy)/(winpercentage/losepercentage))
    print(str(symbol) + ' model Kellys Criterion score is: ' + str(kelly))
    
    # Checklist: Case Based Reasoning: Suggests direction
    # Based on the data input, provide the Buy/Sell signal to Broker Agent
    action = model.predict(data_input)
    action = action[0]
    if action == 1:
      action = 'buy'
    elif action == 0:
      action = 'sell'
    
    print(str(symbol) + ' is recommended to ' + str(action))

    # Put the results of the function for the Broker Agent
    decider_result['kelly']=kelly
    decider_result['action']=action

    return kelly

## Broker Agent


The Broker Agent receives the buy/sell signal and weightage allocation of the Kelly Criterion to execute the trades in the HitBtc demo account through the `do_trade` function by also calling on the `place_order` and `market_order` functions. The Broker Agent is also responsible for all communications with the HitBtc API including `get_current_balances`, `order_history` as if we had a real-life human to help us with all exchange matters. The `get_session` functions enables all the other functions to authorize the session.

In [ ]:
class BrokerAgent():
  # Class variables
  # Fill in your secret and public key here to link to your HITBTC demo account
  public_key = '9Dj54wzii76aPdYsEVg9HsO1CGJdUeuq'
  secret_key = '32IW691CW0X136eBgRYdK_d_cX1uX_tV'
  
  # Defines the sessions for HITBTC in other to get authorized
  def get_session():
    session = requests.session()
    session.auth = (BrokerAgent.public_key, BrokerAgent.secret_key)
    return session

  # Checks the available balances of the trading account for ETH, EOS and USD
  # Checklist: Checks balances available
  def get_current_balances():
    
    balances = BrokerAgent.get_session().get('https://api.demo.hitbtc.com/api/2/trading/balance').json()
    
    current_ETH_balance = next(item for item in balances if item["currency"] == "ETH")['available']
    current_EOS_balance = next(item for item in balances if item["currency"] == "EOS")['available']
    current_USD_balance = next(item for item in balances if item["currency"] == "USD")['available']
    try:
      current_ETH_balance = float(current_ETH_balance)
    except:
      current_ETH_balance = 0
    try:
      current_EOS_balance = float(current_EOS_balance)
    except:
      current_EOS_balance = 0
    try:
      current_USD_balance = float(current_USD_balance)
    except:
      current_USD_balance = 0

    # additional_ETH_balance = next(item for item in balances if item["currency"] == "ETH")['reserved']
    # additional_EOS_balance = next(item for item in balances if item["currency"] == "EOS")['reserved']
    # additional_USD_balance = next(item for item in balances if item["currency"] == "USD")['reserved']
    # additional_ETH_balance = float(additional_ETH_balance)
    # additional_EOS_balance = float(additional_ETH_balance)
    # additional_USD_balance = float(additional_ETH_balance)
    return current_ETH_balance,current_EOS_balance,current_USD_balance

  # Place limit order
  def place_order(symbol,side,types,price,quantity):
    orderData = {'symbol':symbol, 'side': side, 'type' : types, 'price': price, 'quantity':quantity }
    r = BrokerAgent.get_session().post('https://api.demo.hitbtc.com/api/2/order', data = orderData)
    return r.json()

  # Place market order
  def market_order(symbol,side,types,quantity):
    orderData = {'symbol':symbol, 'side': side, 'type' : types, 'quantity':quantity }
    r = BrokerAgent.get_session().post('https://api.demo.hitbtc.com/api/2/order', data = orderData)
    return r.json()

  # Cancel all orders
  def cancel_order():
    r = BrokerAgent.get_session().delete('https://api.demo.hitbtc.com/api/2/order')
    return r.json()

  # Get order history
  # Checklist: Needs to report whether was complete and if fill or partial fil and at what price
  def order_history():
    r = BrokerAgent.get_session().get('https://api.demo.hitbtc.com/api/2/history/order')
    data = r.text
    r_data = json.loads(data)
    rdf = pd.DataFrame(r_data)
    return r.json(), rdf
  
  # Checklist: Call the exchange API with Ticker, Action Sell/buy, Amount, order type to send to the exchange.
  def do_trade(kelly,btcsymbol,binsymbol,current_price,other_price,action,broker_result):
    # Checklist: Receives the decision from the Decider agent
    # If the Decider Agent has computed that Kelly Criterion is negative, we will not go ahead with the bet since the model is telling us not to allocate anything to the bet
    print('######### Broker Agent Start #########')
    kelly = float(kelly)
    # Set the amount of time to sleep
    seconds = 240    
    # If Kelly Criterion is negative, print statement to user and timeout till the next interval
    if kelly <= 0:
      print('no orders executed for ' + btcsymbol + ' due to Kellys Criterion of {:.2f} awaiting next signal from agents...'.format(kelly))
      
      # wait for the next time interval for the next bet
      t_end = time.time() + seconds
      while time.time() < t_end:
        pass
      
      # indicate that no buy or sell was made
      buy_order_id = 0 
      sell_order_id = 0      

    # Else we will act based on the Kelly Criterion to either buy or sell
    else:
      # Extract the current available USD balance
      current_ETH_balance, current_EOS_balance, current_USD_balance = BrokerAgent.get_current_balances()
      total_balance_USD = current_USD_balance

      # Checklist: Quantity limited based on amount available to trade
      max_investment_USD = kelly * total_balance_USD
      
      if action == 'buy':
        try:
          # Agent will attempt to execute a buy order
          print('buying ' + btcsymbol + ' based on agent signal for a maximum recommended USD exposure of ' + str(max_investment_USD))
          # Checklist: Send the decision to the Broker agent.
          order_details = BrokerAgent.place_order(btcsymbol,'buy','limit',current_price,max_investment_USD/current_price)
          buy_order_id = order_details['id']
          print('A ' + order_details['side'] + ' order with id' + str(order_details['id']) + ' has been placed for a quantity of ' + order_details['quantity'] + ' '  + order_details['symbol'] + ' at price of ' + order_details['price'])  
          # a simple variable tracker to note the buy status
          keep_track = 0
        
        except:
          # Print statement to user if unsuccessful
          print('buy order was not successful due to lack of funds or quantity placed too small for '+btcsymbol)
          
          # wait for the next time interval for the next bet
          t_end = time.time() + seconds
          while time.time() < t_end:
            pass

          # indicate that no buy or sell was made
          buy_order_id = 0 
          sell_order_id = 0
          
          # a simple variable tracker to note the buy status
          keep_track = 2
        
        print('You may view your active orders here on the first row here: https://docs.google.com/spreadsheets/d/1uJKZmnutuXwjMWl3sZ0GjJWprb_x9Cpf8kM2XdRIPUQ/edit#gid=0')
        
        # if buy order was successful, we need to monitor the trade within the time interval for take profit or stop loss
        # keep_track == 0 means buy order was successfully submitted 
        if keep_track == 0:

          # wait for the next time interval for the next bet
          t_end = time.time() + seconds - 120
          while time.time() < t_end:
            time.sleep(5)
            # extracting the current market price from a liquid market to check for stop loss or take profit
            url='https://api.binance.com/api/v3/ticker/price?symbol='
            api = url + binsymbol
            api = requests.get(api).json()
            price_now = api['price']
            price_now = float(price_now)
                      
            # extract the order history for the user to see
            sheet = wb.worksheet('Orders')
            rdf = BrokerAgent.order_history()[1]
            PNLAgent.pandas_to_sheets(rdf,sheet)
            
            print('monitoring the bet signal within the time interval for stop loss and take profit for ... '+btcsymbol)
            # Checklist: Consider whether to close positions
            # 20% take profit
            if price_now > current_price*1.2:
              # Get the current available broker balances for the EOS or ETH
              if btcsymbol == 'ETHUSDT':
                current_balance = BrokerAgent.get_current_balances()[0]
              else:
                current_balance = BrokerAgent.get_current_balances()[1]
              
              print('20% take profit reached, initiating sell signal for '+btcsymbol)
              try:
                sale = BrokerAgent.market_order(btcsymbol,'sell','market',current_balance)
                print('A ' + sale['side'] + ' order with id' + str(sale['id']) + ' has been placed for'  + sale['symbol'])
              except:
                print('agent will try to initiate sell order next round, sell order was not successful due to quantity placed too small for '+btcsymbol)
              time.sleep(3)
              try:
                sell_order_id = sale['id']
                print('sell order successful for '+btcsymbol)
              except:
                # indicate that no sell was made
                sell_order_id = 0
              # a simple variable tracker to note the trade status
              keep_track = 1
              break
          
            # Checklist: Consider if goal of company has been achieved so can take a safe position (i.e. it has achieved the amount of profit or much more than planned so consider to stop trading too)
            # 20% stop loss  
            elif price_now < current_price*0.8:
              # Get the current available broker balances for the EOS or ETH
              if btcsymbol == 'ETHUSDT':
                current_balance = BrokerAgent.get_current_balances()[0]
              else:
                current_balance = BrokerAgent.get_current_balances()[1]
              time.sleep(3)
              print('20% stop loss reached, initiating sell signal for '+btcsymbol)
              try:
                sale = BrokerAgent.market_order(btcsymbol,'sell','market',current_balance)
                print('A ' + sale['side'] + ' order with id' + str(sale['id']) + ' has been placed for'  + sale['symbol'])
              except:
                print('agent will try to initiate sell order next round, sell order was not successful due to quantity placed too small for '+btcsymbol)
              time.sleep(3)
              try:
                sell_order_id = sale['id']
                print('sell order successful for '+btcsymbol)
              except:
                # indicate that no sell was made
                sell_order_id = 0
              
              # a simple variable tracker to note the trade status
              keep_track = 1
              break

            # Do nothing if both profit and loss thresholds are not reached
            else:
              pass
          
          # At the end if nothing happens try to sell the order at the original price to prevent risk of holding onto the asset
          # keep_track == 1 means take profit or stop loss triggered hence no need to square off 
          if keep_track == 1:
            pass
          else:
            # Get the current available broker balances for the EOS or ETH
            if btcsymbol == 'ETHUSDT':
              current_balance = BrokerAgent.get_current_balances()[0]
            else:
              current_balance = BrokerAgent.get_current_balances()[1]
            
            print("a market sale order has been put to square off the bet for "+btcsymbol)
            
            try:
              sale = BrokerAgent.market_order(btcsymbol,'sell','market',current_balance)
              print('A ' + sale['side'] + ' order with id' + str(sale['id']) + ' has been placed for'  + sale['symbol'])
            except:
              print('agent will try to initiate sell order next round, sell order was not successful due to quantity placed too small for '+btcsymbol)
            time.sleep(3)
            try:
              sell_order_id = sale['id']
              print('sell order successful for '+btcsymbol)
            except:
              # indicate that no sell was made
              sell_order_id = 0
        
        # keep_track != 0 means buy order was not successfully submitted 
        else:
          pass
      
      # If the signal is to sell
      else:
        print('signal was to sell ' + btcsymbol + ', attempting to sell any on hand and preparing for next signal...')
        
        # indicate that no buy was made
        buy_order_id = 0 
        
        # Get the current available broker balances for the EOS or ETH
        if btcsymbol == 'ETHUSDT':
          current_balance = BrokerAgent.get_current_balances()[0]
        else:
          current_balance = BrokerAgent.get_current_balances()[1]
        
        try:
          sale = BrokerAgent.market_order(btcsymbol,'sell','market',current_balance)
          print('A ' + sale['side'] + ' order with id' + str(sale['id']) + ' has been placed for '  + sale['symbol'])
        except:
          print('agent will try to initiate sell order next round, sell order was not successful due to quantity placed too small for '+btcsymbol)
        time.sleep(3)
        try:
          sell_order_id = sale['id']
          print('sell order successful for '+btcsymbol)
        except:
          # indicate that no sell was made
          sell_order_id = 0

        # wait for the next time interval for the next bet
        t_end = time.time() + seconds
        while time.time() < t_end:
          pass
        
    broker_result['buy'] = buy_order_id
    broker_result['sell'] = sell_order_id
    
    return buy_order_id,sell_order_id

## PNL Agent 

The PNL Agent helps to compute any trades executed by the Broker Agent through the `compute_trade_PNL` function. It will then return the values into a google spreadsheet. Subsequently we utilize Zapier to send out trade transaction results to our human CEO.

In [ ]:
 class PNLAgent():
  
  # A function to process dataframes (mainly from Broker Agent) to find out the number of rows and columns
  def iter_pd(df):
    for val in df.columns:
        yield val
    for row in df.to_numpy():
        for val in row:
            if pd.isna(val):
                yield ""
            else:
                yield val

  # A function to clear an entire sheet then extract pandas dataframes into the sheet
  def pandas_to_sheets(pandas_df, sheet, clear = True):
    if clear:
          sheet.clear()
    (row, col) = pandas_df.shape
    cells = sheet.range("A1:{}".format(gspread.utils.rowcol_to_a1(row + 1, col)))
    for cell, val in zip(cells, PNLAgent.iter_pd(pandas_df)):
        cell.value = val
    sheet.update_cells(cells)

  def compute_trade_PNL(buyorder,sellorder,binsymbol,ID,oldprice,PNL_result):
    print('######### PNL Agent Start #########')
    url='https://api.binance.com/api/v3/ticker/price?symbol='

    if buyorder == 0:
      # If there were no orders fulfilled, print statement to user
      simple_return = 0 
      cumulative_return = 0
      print('There were no PNL updates for ' + str(binsymbol) + ' as no orders were placed')
    
    else:
      # Extract whether the buy status was even partially filled
      order_details = BrokerAgent.order_history()[0]
      buy_status = next(item for item in order_details if item["id"] == buyorder)['status']
      
      if buy_status == 'new':
        # If there were no buy orders fulfilled, print statement to user as other would have been cancelled
        print('There were no PNL updates for ' + str(binsymbol) + ' as buy order was not filled')
        simple_return = 0 
        cumulative_return = 0
      
      else:
        result = []
        # Get the broker's buy details
        buy_time = next(item for item in order_details if item["id"] == buyorder)['updatedAt']
        symbol = next(item for item in order_details if item["id"] == buyorder)['symbol']
        buy_price = float(next(item for item in order_details if item["id"] == buyorder)['avgPrice'])
        buy_quantity = float(next(item for item in order_details if item["id"] == buyorder)['cumQuantity'])
        
        # Get the broker's sell details
        if sellorder == 0:
          print('There is slight asset exposure to ' + str(binsymbol) + ' as there was not enough liquidity in the market. The bot will attempt to square this off in the next bet interval.')
          sell_time = 'NA'
          sell_price = 'NA'
          sell_quantity = 'NA'
        else:
          # Get the broker's sell details
          sell_time = next(item for item in order_details if item["id"] == sellorder)['updatedAt']
          sell_price = float(next(item for item in order_details if item["id"] == sellorder)['avgPrice'])
          sell_quantity = float(next(item for item in order_details if item["id"] == sellorder)['cumQuantity'])
        print('Getting trade details for '+ str(binsymbol) + ' and uploading to https://docs.google.com/spreadsheets/d/1uJKZmnutuXwjMWl3sZ0GjJWprb_x9Cpf8kM2XdRIPUQ/edit#gid=614512911')
        
        # Computing the % returns for each trade
        try:
          theoretical_PNL_percentage = (price_now-buy_price)/buy_price
        except:
          theoretical_PNL_percentage = 0
        theoretical_PNL_dollar = theoretical_PNL_percentage * buy_quantity
        
        try:
          actual_PNL_percentage = (sell_price-buy_price)/buy_price
        except:
          actual_PNL_percentage = 0
        try:
          actual_PNL_dollar = actual_PNL_percentage * sell_quantity
        except:
          actual_PNL_dollar = 0
        
        # Computing the current portfolio balance in USD
        current_ETH_balance, current_EOS_balance, current_USD_balance = BrokerAgent.get_current_balances()
        
        api = url + 'ETHUSDT'
        api = requests.get(api).json()
        price = api['price']
        ETH_price = float(price)

        api = url + 'EOSUSDT'
        api = requests.get(api).json()
        price = api['price']
        EOS_price = float(price)

        total_balance_USD = current_ETH_balance * ETH_price + current_EOS_balance * EOS_price + current_USD_balance
        
        # Calculating returns based on USD 2000 start base
        # Checklist: Consider whether drawdown (realized losses) is too much so stop trading?
        simple_return = (total_balance_USD - 2000) / 2000 # HitBTC Demo accounts start with 2000 USD
        
        date_format = "%Y/%m/%d"
        Project_Current_Date = datetime.now().strftime("%Y/%m/%d")
        global Project_Start_Date
        a = datetime.strptime(Project_Start_Date, date_format)
        b = datetime.strptime(Project_Current_Date, date_format)
        delta = b - a
        if delta.days == 0:
          days_passed = 1
        else:
          days_passed = float(delta.days)
        
        cumulative_return = (total_balance_USD/2000)**(1/(days_passed/365)) - 1 # HitBTC Demo accounts start with 2000 USD    
        
        # Formatting return results
        buy_price = float("{:.2f}".format(buy_price))
        sell_price = float("{:.2f}".format(sell_price))
        buy_quantity = float("{:.4f}".format(buy_quantity))
        sell_quantity = float("{:.4f}".format(sell_quantity))

        total_balance_USD = float("{:.2f}".format(total_balance_USD))
        theoretical_PNL_percentage = float("{:.2f}".format(theoretical_PNL_percentage*100))
        theoretical_PNL_dollar = float("{:.2f}".format(theoretical_PNL_dollar))
        actual_PNL_percentage = float("{:.2f}".format(actual_PNL_percentage*100))
        actual_PNL_dollar = float("{:.2f}".format(actual_PNL_dollar))
        simple_return = float("{:.2f}".format(simple_return*100))
        cumulative_return = float("{:.2f}".format(cumulative_return*100))

        # Passing the PNL results to the PNL sheet
        result=[]
        result.append(buy_time)
        result.append(sell_time)
        result.append(symbol)
        result.append(buyorder)
        result.append(buy_price)
        result.append(buy_quantity)
        result.append(sellorder)
        result.append(sell_price)
        result.append(sell_quantity)
        result.append(theoretical_PNL_percentage)
        result.append(theoretical_PNL_dollar)
        result.append(actual_PNL_percentage)
        result.append(actual_PNL_dollar)
        result.append(total_balance_USD)
        result.append(simple_return)
        result.append(cumulative_return)
        result.append(ID)
        ws = wb.worksheet('PnL')
        ws.append_row(result)
        
        # data_agent = DataAgent.getInstance()
        # data_agent.write_to_pnl_worksheet(result)
    
    # Get the current price details to label for the next KNN
    
    api = url + binsymbol
    api = requests.get(api).json()
    price_now = api['price']
    price_now = float(price_now)

    # Update cases label for KNN, 1 is buy 0 is sell
    if float(price_now) > float(oldprice):
      label = 1
    else:
      label = 0
    
    # Outputing the case label for KNN
    PNL_result['label']=label

    # Updating all the trade history
    try:
      sheet = wb.worksheet('Orders')
      rdf = BrokerAgent.order_history()[1]
      PNLAgent.pandas_to_sheets(rdf,sheet)
    except:
      pass

## PowerBI Agent

The `PowerBI_Agent` function reads in PnL results from our Multi-Agent System and uploads the datasets to PowerBI for data visualization. Subsequently we utilize Zapier to send out data visualizations for our PowerBI results to our human CEO.

In [ ]:
def PowerBI_Agent():
  print('######### PowerBI Agent Start #########')
  global cdf
  url = 'https://api.powerbi.com/beta/5ba5ef5e-3109-4e77-85bd-cfeb0d347e82/datasets/becd0daa-1b92-48fd-b0fe-ff6d88627093/rows?noSignUpCheck=1&key=wCOvMha1nbVwCqHq6%2B7WPIQful7qVhZBPjSMrWqAA5t4ONSYXN8bRwhQf3H2QXRMGyk%2Fw01O84Z6jZ4%2F8zMipg%3D%3D'
  # Function called fetch_output with input cfg to check what datasets have already been uploaded to PowerBi. Returns new datasets that have not been uploaded.
  def fetch_output(cfg):
    json_file=('/content/Group3_database_secretkeye.json')
    # Define the scope
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

    # Add the credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name(json_file, scope)

    # Authorize the clientsheet 
    client = gspread.authorize(creds)

    # Open the Google Sheet named IS5006_DB which is constantly updated every 5mins by PnL Agent
    sheet = client.open('IS5006_DB').worksheet("PnL")

    # Retrieve data from the Google Sheet and assign to Dataframe df
    records_data = sheet.get_all_records()
    rows = sheet.get_all_values()
    df=pd.DataFrame.from_records(rows)
    df.columns = df.iloc[0]
    df = df.iloc[1:]
    df.describe()
  
    # Check if any data in DataFrame df_is has been uploaded to powerBI before with cdf
    # If they are already uploaded, the data is dropped from df
    for index,row in df.iterrows():
      if (cdf[cdf['Buy Order Start']==row['Buy Order Start']].empty):
        print('Data not updated into PowerBi before.')
      else:
        if (row['Buy Order Start'] in cdf[cdf['Symbol']==row['Symbol']]['Buy Order Start'].values):
          print('Data already updated into PowerBi.')
          df=df.drop(index=index)
    return df

  #Check if cdf dataframe already existed from previous runs
  if 'cdf' in globals():
    if isinstance(cdf,pd.DataFrame):
      # print('hi')
      # Fetch the ouput from the database generated from PnL Agent
      result=fetch_output(cdf)
    # Else create a new dataframe called cdf
    else :
      cdf = pd.DataFrame()
      cdf['Buy Order Start'] = ""
      cdf['Symbol'] = ""
      # Fetch the ouput from the database generated from PnL Agent
      result=fetch_output(cdf)
  else :
    cdf = pd.DataFrame()
    cdf['Buy Order Start'] = ""
    cdf['Symbol'] = ""
    # Fetch the ouput from the database generated from PnL Agent
    result=fetch_output(cdf)
  # Append the results from database to cdf. This dataframe is use to check if data has been already uploaded to PowerBi
  cdf=cdf.append(result)
  # print(cdf)
  df=result
  # print(result)
  # If there are no new datasets pass
  if df.empty:
    pass
  else:
  # If there are new datasets, upload to PowerBi
    for ind in df.index:
      Date=df['Buy Order Start'][ind]
      Sym=df['Symbol'][ind]
      tp=df['Theoretical PNL (%)'][ind]
      tusd=df['Theoretical PNL (USD)'][ind]
      apnl=df['Actual PNL (%)'][ind]
      apnlusd=df['Actual PNL (USD)'][ind]
      bal=df['Ending Balance (USD)'][ind]
      ror=df['Rate of Return %'][ind]
      par=df['P.A Return %'][ind]
      data = [
        {
        "Date (UTC)" :Date,
        "Symbol" :Sym,
        "Theoretical PNL (%)" :tp,
        "Theoretical PNL (USD)" :tusd,
        "Actual PNL (%)" :apnl,
        "Actual PNL (USD)" :apnlusd,
        "Balance (USD)" :bal,
        "Rate of Return (%)" :ror,
        "P.A Return (%)" :par,
        }
        ]
      # print(data)
      
      headers = {
        "Content-Type": "application/json"
        }
      response = requests.request(
      method="POST",
      url=url,
      headers=headers,
      data=json.dumps(data)
      )

## CEO

The CEO is meant to organize all the other agents through multi-threading. Killing the code would mean a hard stop on the CBR. This could happen for example if the real-life human CEO receives live updates of the trades via zapier and notices the return % dropping drastically. This code is meant to otherwise be ran on an infinite loop

In [ ]:
  # setup
data_agent = DataAgent.getInstance() 
data_agent.execute()
isRunning = True
manager = Manager()
eth_result = manager.dict()
eos_result = manager.dict()
count = 1
while True:
    print("Cycle", count)
    count+=1 
    timenow = datetime.utcnow()

    crossoverDecision_process = Process(target = CrossoverAgent().run, args = (eos_result, eth_result))
    crossoverDecision_process.start()

    commmodities_process = Process(target = CommoditiesAgent, args = (eos_result, eth_result))
    commmodities_process.start()

    twitter_process = Process(target = TwitterAgent().fetch_tweets, args = (timenow, Constants.EOS, eos_result))
    twitter_process.start()

    twitter_process2 = Process(target = TwitterAgent().fetch_tweets, args = (timenow, Constants.ETH, eth_result))
    twitter_process2.start()

    crossoverDecision_process.join()
    commmodities_process.join()
    twitter_process2.join()

    print("########### N Agents Complete ##############")

    # Processing values from N Agents
    manager = Manager()
    eth_decider_result = manager.dict()
    eos_decider_result = manager.dict()
    eth_broker_result = manager.dict()
    eos_broker_result = manager.dict()
    eth_pnl_result = manager.dict()
    eos_pnl_result = manager.dict()

    ETH_input = list(eth_result.values())
    ETHprice_current = ETH_input[3]
    ETHMA_current = ETH_input[2]
    ETHaverage_subjectivity = ETH_input[1]
    ETHaverage_polarity = ETH_input[0]
    ETHcommodity_current = ETH_input[4]
    ETHinput = pd.DataFrame({'MAAgent':[ETH_input[2]],'AverageSubjectivity':[ETH_input[1]], 'AveragePolarity':[ETH_input[0]],'CommoditiesAgent':[ETH_input[4]]})

    EOS_input = list(eos_result.values())
    EOSprice_current = EOS_input[3]
    EOSMA_current = EOS_input[2]
    EOSaverage_subjectivity = EOS_input[1]
    EOSaverage_polarity = EOS_input[0]
    EOScommodity_current = EOS_input[4]
    EOSinput = pd.DataFrame({'MAAgent':[EOS_input[2]],'AverageSubjectivity':[EOS_input[1]], 'AveragePolarity':[EOS_input[0]],'CommoditiesAgent':[EOS_input[4]]})

    timenow = datetime.utcnow()
    # auth.authenticate_user()
    # gc = gspread.authorize(GoogleCredentials.get_application_default()) 
    scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        '/content/client_secret_2.json',
        scopes=scopes
    )

    gc = gspread.authorize(credentials)

    wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1uJKZmnutuXwjMWl3sZ0GjJWprb_x9Cpf8kM2XdRIPUQ/edit#gid=0')

    # Decider Threading
    ETHdecider_process = Process(target = DeciderAgent.run_latest_model, args = (Constants.ETH,ETHinput,ETHprice_current,eth_decider_result))
    ETHdecider_process.start()
    ETHID = Recommendation_ID
    Recommendation_ID +=1
    print('ETH recommendation ID is ' + str(Recommendation_ID-1))

    EOSdecider_process = Process(target = DeciderAgent.run_latest_model, args = (Constants.EOS,EOSinput,EOSprice_current,eos_decider_result))
    EOSdecider_process.start()
    EOSID = Recommendation_ID
    Recommendation_ID +=1
    print('EOS recommendation ID is ' + str(Recommendation_ID-1))

    ETHdecider_process.join()
    EOSdecider_process.join()

    # Processing values from Decider Agent to Broker Agent
    ETH_decider = list(eth_decider_result.values())
    ETHkelly = ETH_decider[0]
    ETHaction = ETH_decider[1]

    EOS_decider = list(eos_decider_result.values())
    EOSkelly = EOS_decider[0]
    EOSaction = EOS_decider[1]

    # Broker Threading
    ETHbroker_process = Process(target = BrokerAgent.do_trade, args = (ETHkelly,'ETHUSDT','ETHUSDT',ETHprice_current, EOSprice_current, ETHaction, eth_broker_result))
    ETHbroker_process.start()

    EOSbroker_process = Process(target = BrokerAgent.do_trade, args = (EOSkelly,'EOSUSD','EOSUSDT',EOSprice_current, ETHprice_current, EOSaction, eos_broker_result))
    EOSbroker_process.start()

    ETHbroker_process.join()
    EOSbroker_process.join()

    # Processing values from Broker Agent to PNL Agent
    ETH_broker = list(eth_broker_result.values())
    ETHbuyID = ETH_broker[0]
    ETHsellID = ETH_broker[1]

    EOS_broker = list(eos_broker_result.values())
    EOSbuyID = EOS_broker[0]
    EOSsellID = EOS_broker[1]

    # PnL Threading
    ETHPnl_process = Process(target = PNLAgent.compute_trade_PNL, args = (ETHbuyID,ETHsellID,'ETHUSDT',ETHID, ETHprice_current,eth_pnl_result))
    ETHPnl_process.start()

    EOSPnl_process = Process(target = PNLAgent.compute_trade_PNL, args = (EOSbuyID,EOSsellID,'EOSUSDT',EOSID, EOSprice_current,eos_pnl_result))
    EOSPnl_process.start()

    ETHPnl_process.join()
    EOSPnl_process.join()

    BrokerAgent.cancel_order()

    # Processing values from PNL Agent to CEO
    ETH_pnl = list(eth_pnl_result.values())
    ETHlabel = ETH_pnl[0]

    EOS_pnl = list(eos_pnl_result.values())
    EOSlabel = EOS_pnl[0]

    # Update Cases
    ETHDatabase = []
    ETHDatabase.append(str(timenow))
    ETHDatabase.append(ETHprice_current)
    ETHDatabase.append(ETHMA_current)
    ETHDatabase.append(ETHaverage_subjectivity)
    ETHDatabase.append(ETHaverage_polarity)
    ETHDatabase.append(reduce(lambda x,y: x+" "+y, ETHcommodity_current))
    ETHDatabase.append(ETHlabel)
    # data_agent.write_to_specific_worksheet(data_agent.eth_db_worksheet, ETHDatabase)
    print("Writing to ETHDatabase Sheet in Google Sheet")
    ws = wb.worksheet('ETHDatabase')
    ws.append_row(ETHDatabase)
    print("Writing complete...")

    EOSDatabase = []
    EOSDatabase.append(str(timenow))
    EOSDatabase.append(EOSprice_current)
    EOSDatabase.append(EOSMA_current)
    EOSDatabase.append(EOSaverage_subjectivity)
    EOSDatabase.append(EOSaverage_polarity)
    EOSDatabase.append(reduce(lambda x,y: x+" "+y, EOScommodity_current))
    EOSDatabase.append(EOSlabel)
    # data_agent.write_to_specific_worksheet(data_agent.eos_db_worksheet, EOSDatabase)
    print("Writing to EOSDatabase Sheet in Google Sheet")
    ws = wb.worksheet('EOSDatabase')
    ws.append_row(EOSDatabase)
    print("Writing complete...")

    print('##### A bet interval has been completed #####')
    print('You may view the latest transactions and gain/loss at https://docs.google.com/spreadsheets/d/1uJKZmnutuXwjMWl3sZ0GjJWprb_x9Cpf8kM2XdRIPUQ/edit#gid=614512911')
    
    print('Pushing data to PowerBI...')
    PowerBI_Agent()
    DataAgent.cleanup()
    print('cleaning up instance....')

=========Pulling EOS data========
extracting data from 04/11/2021, 02:20:00 to 04/11/2021, 04:33:28
2021-04-11 02:20:00
2021-04-11 04:33:28.505522
=========Pulling ETH data========
extracting data from 04/11/2021, 02:20:00 to 04/11/2021, 04:33:30
2021-04-11 02:20:00
2021-04-11 04:33:30.016989
Cycle 1
Doing crossover analysis
Twitter Agent's Output To Google Sheets
averagePolarityScore 0.13636363636363635 averageSubjectivityScore 0.5
Twitter Agent's Output To Google Sheets
averagePolarityScore 0 averageSubjectivityScore 0
MA Agent's Output To Google Sheets
Close Price 6.8391 MA Agent 0.0
MA Agent's Output To Google Sheets
Close Price 2138.4 MA Agent 0.0
Commodities Agent's Output To Google Sheets
ETH 0.03273913550040778 EOS 0.010369334731445882
########### N Agents Complete ##############
ETH recommendation ID is 62
######### Decider Agent Start #########
######### Decider Agent Start #########
EOS recommendation ID is 63
EOS model Accruacy score is: 0.5033333333333333
EOS model Precisi

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


EOS model Accruacy score is: 0.46511627906976744
EOS model Precision score is: 0.5614035087719298
EOS model Kellys Criterion score is: 0.1355223451186146
EOS is recommended to sell
ETH model Accruacy score is: 0.49776119402985075
ETH model Precision score is: 0.49818181818181817
ETH model Kellys Criterion score is: -0.013382487026163181
ETH is recommended to buy
######### Broker Agent Start #########
no orders executed for ETHUSDT due to Kellys Criterion of -0.01 awaiting next signal from agents...
######### Broker Agent Start #########
signal was to sell EOSUSD, attempting to sell any on hand and preparing for next signal...
agent will try to initiate sell order next round, sell order was not successful due to lack of balance or quantity placed too small for EOSUSD
